In [1]:
import asyncio
import re
import time
import pandas as pd
import numpy as np
from playwright.async_api import async_playwright
from sklearn.feature_extraction.text import TfidfVectorizer
import aiohttp        
import aiofiles
import time
import os.path
from tqdm.notebook import tqdm

SELECTING WORDS

In [2]:
base_path = '../data'
# base_path = 'data'
train_df = pd.read_csv(base_path + '/train.csv')


In [3]:
train_df

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069
...,...,...,...,...,...
34245,train_4028265689,fff1c07ceefc2c970a7964cfb81981c5.jpg,e3cd72389f248f21,Masker Bahan Kain Spunbond Non Woven 75 gsm 3 ...,3776555725
34246,train_769054909,fff401691371bdcb382a0d9075dfea6a.jpg,be86851f72e2853c,MamyPoko Pants Royal Soft - S 70 - Popok Celana,2736479533
34247,train_614977732,fff421b78fa7284284724baf249f522e.jpg,ad27f0d08c0fcbf0,KHANZAACC Robot RE101S 1.2mm Subwoofer Bass Me...,4101248785
34248,train_3630949769,fff51b87916dbfb6d0f8faa01bee67b8.jpg,e3b13bd1d896c05c,"Kaldu NON MSG HALAL Mama Kamu Ayam Kampung , S...",1663538013


In [83]:
ge5 = train_df.groupby(['label_group']).filter(lambda x: x['title'].size >= 5)

In [84]:
ge5

,posting_id,image,image_phash,title,label_group
6,train_1802986387,00144a49c56599d45354a1c28104c039.jpg,f815c9bb833ab4c8,Jubah anak size 1-12 thn,1835033137
15,train_4196427721,002039aaf8618627a0442d5e89e5dda6.jpg,e98c873acc65946e,Korek Kuping LED untuk balita CherryBabyKidsSh...,349297863
19,train_2961381387,00303ad1c062fdeaf5f41b9ffb71a5fb.jpg,e48d9b652098efe1,Madame Gie MakeUp Blush On BY GISELL,2098400894
22,train_1396146683,0035b8fc42955c0fa2301b68e2ffe861.jpg,d5c0caa0aeb3b23e,CHOCO BALL LAGIE COKLAT Lagie Grosir Cokelat L...,4240123481
24,train_1872347548,003906c6c6bba4cde15e5e854dba8f6d.jpg,f3e18c5ea14493e3,Stand Hanger Multifungsi - Tiang Gantungan Baj...,4044205497
...,...,...,...,...,...
34210,train_3738631799,ffa7e451b9a6f15060351316c731d057.jpg,ea9dc137c24a8e63,"b""DRESS MUSLIM MURAH HAYLA DRESS '""",3861748824
34217,train_2051967800,ffb88cdb27676ed9f142a6ee615b6452.jpg,e9dccb5c421f1638,Lampu Tumblr agustusan warna warni bisa disamb...,1018873174
34218,train_3349791994,ffb9b59c7669c781847fba7499bd7699.jpg,f4ce05c51f71705a,LACHEL VITAMIN C LACHEL VIT C GLUTA FROZEN COL...,1911010635
34235,train_3668806308,ffdd054398884b04bdab9e41d30c7fb2.jpg,9ba6cc38a4a792f1,MAYCREATE MOISTURIZING SPRAY 150ML(1KG 6PCS) ...,1141798720


In [3]:
groups = train_df.groupby(['label_group']).filter(lambda x: x['title'].size < 6)
labels = groups['label_group'].unique().tolist()


In [4]:
groups['title'] = [ s.encode('latin1').decode('unicode-escape').encode('latin1').decode('utf-8') for s in groups['title']]
groups['title'] = groups.title.str.lower()
groups['title'] = [re.sub('[^0-9a-zA-Z ,.]+', ' ', x) for x in groups['title']]

In [5]:
# groups['len_words'] = [len(re.sub('[^0-9a-zA-Z ,.]+', ' ', x).split()) for x in groups['title']]

In [5]:
grouped = groups.groupby('label_group')

In [6]:
products = []
for id in labels:
    product = {}
    curr = grouped.get_group(id)
    product['label_group'] = id
    base = r'^{}'
    expr = '(?=.*{})'
    v = TfidfVectorizer()
    x = v.fit_transform(curr['title'])
    y = v.transform([' '.join(curr["title"])])
    feature_array = v.get_feature_names_out()
    if feature_array.shape[0] > 5:
        tfidf_sorting = np.argsort(y.toarray()).flatten()[::-1]
        top_n = feature_array[tfidf_sorting][:5].tolist()
        product['query'] = ('https://shopee.co.id/search?keyword=' + " ".join(top_n)).replace(" ", "%20")
        
        product['words'] = base.format(''.join(expr.format(w) for w in top_n))
    else:
        product['query'] = ('https://shopee.co.id/search?keyword=' + curr['title'].iloc[0] ).replace(" ", "%20") 
       
        product['words'] =  base.format(''.join(expr.format(w) for w in feature_array.tolist()))
    products.append(product)

SCRAPING

In [7]:
async def download_img(url,title,label_group):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            if resp.status == 200:
                # start = time.time()
                f = await aiofiles.open(title, mode='wb+')
                await f.write(await resp.read())
                await f.close()
                # end = time.time()
                # print(f"{title} at {label_group} downloaded for {end-start} secs")
                return True
            else:
                return False


async def get_info(item,label_group,words):
    res = dict()
    title = await item.get_attribute('alt')
    matched = (re.sub('[^0-9a-zA-Z ,]+', ' ', title)).lower()
    if re.match(words, matched):
        link = await item.get_attribute('src')
        url = link[:-3]
        name = re.sub('[^0-9a-zA-Z ,]+', ' ', title)
        name = " ".join(name.split()[:6])
        name = base_path + '/scrap_image/' + name + "_" + str(np.random.randint(0, 60)) + "_" + str(label_group)  + '.jpg'
        name = name.replace(' ', "_")
        down = await download_img(url,name,label_group)
        if down:
            res['url'] = url
            res['image'] = name
            res['title'] = title
            res['label_group'] = label_group
            return res
        else:
            res['url'] = url
            res['image'] = name
            res['title'] = title
            res['label_group'] = label_group
            return (res)

async def scrape_category(product,timeout=30000):
    async with async_playwright() as pw:
        # start = time.time()
        browser = await pw.chromium.launch( headless=True)
        page = await browser.new_page()
        await page.goto(product['query'])
        # start_wait = time.time()
        res = await page.wait_for_selector('.shopee-search-item-result', timeout=timeout)
        end_wait = time.time()
        # print(f"product {product['label_group']} waiting page for {end_wait-start_wait} secs")
        for i in range(3):
            await page.wait_for_selector('._7DTxhh.vc8g9F')
            await page.keyboard.down('End')
        selector = '._7DTxhh.vc8g9F'
        all_items = await res.query_selector_all(selector)
        tasks = []
        for item in all_items:
            tasks.append(asyncio.create_task(get_info(item,product['label_group'],product['words'])))
        results = await asyncio.gather(*tasks)
        await browser.close()
        # end = time.time()
        # print(f"product {product['label_group']} got {len(results)} items for {end-start} secs")
        # print(f'Download time: {end - end_wait}, { (end - end_wait) / len(results)} per items')
        return results

async def scraper(products, timeout=30000):
    tasks = []
    for product in products:
        tasks.append(asyncio.create_task(scrape_category(product,timeout=timeout)))
    result = await asyncio.gather(*tasks)
    return result

In [8]:
old_error_list = [58, 71, 103, 104, 116, 131, 156,178, 247, 253 , 277, 368, 412, 422, 424, 425, 461, 463, 475, 476, 481, 528, 541, 555, 577, 589, 624, 684, 691, 733, 759, 787, 789, 838, 843, 847, 871, 878, 879, 904, 919, 965, 980, 987, 995, 1021, 1029, 1030, 1039, 1104, 1178]

In [9]:
async def scrape_many_threads(products,result=[],start=0,end=10088,max_threads=2,max_timeout=60000):
    first_start = start
    prev_start = 0
    ok_count = 0
    error_list = []
    error_count = 0
    threads = min_threads = 1
    timeout = min_timeout = 30000
    while start < end - 1:
        try:
            for i in tqdm(range(start,end,threads)):
                prev_start = start
                if i + threads < end -1:
                    res= await scraper(products[i:i+threads],timeout=timeout)
                else:
                    res= await scraper(products[i:], timeout=timeout)
                result = result +  [x for y in res for x in y if type(x) is dict]
                start = i + threads
                ok_count += 1
                if start % 2 == 0 or start % 3 == 0 or start % 5 == 0:
                    test = pd.DataFrame.from_dict(result)
                    test.to_csv(base_path + '/temp/scrap_temp_s_'+ str(first_start)+ "_" + str(start) + '.csv')
                if ok_count > 0 and ok_count % 25 == 0:
                    error_count = 0
                    timeout = min(max_timeout,timeout * 2)
                    threads = min(max_threads, threads * 2)
                    break
        except KeyboardInterrupt:
            test = pd.DataFrame.from_dict(result)
            test.to_csv(base_path + '/temp/scrap_temp_s_'+ str(first_start)+ "_" + str(start) + '.csv')
            print(start)
            print(error_list)
            return result, error_list
        except:
            if prev_start == start:
                ok_count = 0
                error_count += 1
            print(f'start = {start} threads = {threads}, timeout = {timeout}, error= {error_count}')
            if error_count >= 3 or (timeout == max_timeout and threads == min_threads):
                error_count = 0
                error_list.append(start)
                start = start + 1
                timeout = min_timeout
                threads = min_threads
                print(error_list)
                continue
            if  error_count >= 2 or timeout == max_timeout:
                threads = max (threads // 2, min_threads)
                continue
            if timeout < max_timeout:
                timeout = min(timeout * 1.5, max_timeout)
                continue

    test = pd.DataFrame.from_dict(result)
    test.to_csv(base_path + '/scrap.csv')
    return result, error_list

In [ ]:
results, error_list = await scrape_many_threads(products,start=1817,max_threads=12,max_timeout=60000)

In [10]:
# async def scrape_many(products,result=[],start=0,end=10088):
#     prev_start = 0
#     error_count = 0
#     error_list = []
#     def_timeout  = timeout = 30000
#     while start < end - 1:
#         try:
#             for i in tqdm(range(start,end)):
#                 prev_start = start
#                 res = await scrape_category(products[i],timeout)
#                 result = result +  [x for x in res if type(x) is dict ]
#                 start = i + 1
#                 if i % 10 == 0 or i % 3 == 0 :
#                     test = pd.DataFrame.from_dict(result)
#                     test.to_csv(base_path + '/temp/scrap_temp_s_' + str(start) + "_" + str(time.time()) + '.csv')
#         except KeyboardInterrupt :
#             test = pd.DataFrame.from_dict(result)
#             test.to_csv(base_path + '/temp/scrap_temp_s_' + str(start) + "_" + str(time.time()) + '.csv')
#             print(start)
#             print(error_list)
#             return result, error_list
#         except :
#             if prev_start == start:
#                 error_count += 1
#             if timeout < 60000:
#                 timeout = min(timeout * 1.5, 60000)
#             if error_count > 3 or timeout == 60000:
#                 error_count = 0
#                 error_list.append(start)
#                 start = start + 1
#                 timeout = def_timeout
#             continue
#     test = pd.DataFrame.from_dict(result)
#     test.to_csv(base_path + '/scrap.csv')
#     return result, error_list

In [ ]:
# results, error_list = await scrape_many(products,start=272)

In [ ]:
results, error_list = await scrape_many(products,start=272)

In [4]:
data_1 = pd.read_csv(base_path + '/temp/scrap_temp_s_0_160.csv')
data_2 = pd.read_csv(base_path + '/temp/scrap_temp_s_161_276.csv')
data_3 = pd.read_csv(base_path + '/temp/scrap_temp_s_277_1816.csv')
data_4 = pd.read_csv(base_path + '/temp/scrap_temp_s_1817_4045.csv')

In [5]:
data_1['image'] = data_1['image'].str.removeprefix("../data/scrap_image/")
data_2['image'] = data_2['image'].str.removeprefix("../data/scrap_image/")
data_3['image'] = data_3['image'].str.removeprefix("../data/scrap_image/")
data_4['image'] = data_4['image'].str.removeprefix("../data/scrap_image/")

In [6]:
result = pd.concat([data_1,data_2,data_3,data_4])

In [7]:
result['check'] = base_path + '/train_images/scrape_image/' + result['image']  
result = result[result['check'].map(os.path.exists)]

In [9]:
result = result[['image','title','label_group']]

In [10]:
result['title'].replace("[^0-9a-zA-Z ,.']+",' ',regex=True,inplace=True)

In [11]:
result.to_csv(base_path + '/combined_scrape.csv')

In [12]:
result['image'] = "scrape_image/" + result['image']

In [13]:
concat_train = train_df[['image','title','label_group']]

In [15]:
combined_w_train = pd.concat([concat_train, result])

In [16]:
combined_w_train.to_csv(base_path + '/train_appended.csv')

In [17]:
group_1 = combined_w_train.groupby(['label_group']).filter(lambda x: x['title'].size >= 5)

In [18]:
group_2 = combined_w_train.groupby(['label_group']).filter(lambda x: x['title'].size < 5)

In [19]:
regroup = pd.concat([group_1, group_2])

In [20]:
regroup.reset_index(drop=True,inplace=True)

In [21]:
regroup[['image','title','label_group']].to_csv(base_path + '/appended_train.csv')

In [22]:
for_embedding = regroup

In [23]:
for_embedding['temp'] = for_embedding['image'].str.lower()

In [24]:
for_embedding = for_embedding.groupby(['temp']).filter(lambda x: x['title'].size < 2)

In [25]:
for_embedding.reset_index(drop=True,inplace=True)

In [26]:
for_embedding = for_embedding[['image','title','label_group']]

In [27]:
for_embedding.to_csv(base_path + '/appended_embedding.csv')